
the thing is using .option("startingOffsets", "earliest") option is crucial because by default spark does not read data from begining when it is reading from kafka .... 
so we need to specify the spark to read from start in kafka by the above mentioned option

In [0]:
dbutils.fs.ls('dbfs:/user/hive/warehouse/')

Out[1]: [FileInfo(path='dbfs:/user/hive/warehouse/mydb.db/', name='mydb.db/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/user/hive/warehouse/mytable/', name='mytable/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/user/hive/warehouse/sbit_db.db/', name='sbit_db.db/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/user/hive/warehouse/tararyen/', name='tararyen/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/user/hive/warehouse/targaryen/', name='targaryen/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/user/hive/warehouse/targaryen0/', name='targaryen0/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/user/hive/warehouse/targaryen1/', name='targaryen1/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/user/hive/warehouse/targaryen5/', name='targaryen5/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/user/hive/warehouse/targaryen6/', name='targaryen6/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/user/hive/warehouse/targaryend/', name='tar

In [0]:
%sql
create table if not exists targaryenTemp1(
  key BINARY,
  value BINARY,
  topic STRING,
  partition INT,
  offset LONG,
  timestamp TIMESTAMP,
  timestampType INT
) using delta

In [0]:
kafka_server = "pkc-4r087.us-west2.gcp.confluent.cloud:9092"
kafka_api_secret = "SYRj4UPfB0qXJcYL32B+9FZYgT+SPsSlSyLA3Ww26v7NRxxLxBSdgP6A5Nqq842j"
base_dir_checkpoint = "/project_stuff/checkpoint/"
kafka_api_key = "PWEUCUXA4575GXRN"

df = (spark.readStream.format("kafka") 
     .option("kafka.bootstrap.servers", kafka_server) 
     .option("startingOffsets", "earliest")
     .option("kafka.security.protocol", "SASL_SSL") 
     .option("kafka.sasl.jaas.config", "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username='{}' password='{}';".format(kafka_api_key, kafka_api_secret)) 
     .option("kafka.ssl.endpoint.identification.algorithm", "https") 
     .option("kafka.sasl.mechanism", "PLAIN") 
     .option("subscribe","user_info_topic") 
     .load()
    )

In [0]:
def upsert(df_micro_batch, batch_id):
    df_micro_batch.createOrReplaceTempView("user_delta")
    query = f"""
            MERGE INTO default.targaryenTest a
            USING user_delta b
            ON a.key = b.key
            WHEN NOT MATCHED THEN INSERT *
            """
    print(df_micro_batch.count())
    df_micro_batch._jdf.sparkSession().sql(query)

In [0]:
stream_writer = (df.writeStream 
                                 .format("delta")
                                #  .foreachBatch(upsert) 
                                 .option("checkpointLocation", "/gymgcs8jgjjgvjgchv1hvkyhvlc055test") 
                                 .outputMode("append") 
                                 .queryName("gym_logikhctkjhg1vhgjkhvhvcstream")
                )

In [0]:
stream_writer.trigger(processingTime="10 minutes").toTable('default.targaryenTemp1')

Out[36]: <pyspark.sql.streaming.query.StreamingQuery at 0x7f5c442b1460>

In [0]:
%sql
select count(*) from default.targaryenTemp1

count(1)
13
